# Customer Segmentation Analysis

This notebook contains the analysis and modeling for customer segmentation using machine learning techniques.


## 1. Import Libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")


## 2. Load Data


In [ ]:
# Load your customer data
# df = pd.read_csv('your_data.csv')

# Display basic information
# print(df.head())
# print(df.info())
# print(df.describe())


## 3. Data Exploration


In [ ]:
# Explore your data
# Check for missing values
# df.isnull().sum()

# Visualize distributions
# df.hist(bins=30, figsize=(15, 10))
# plt.tight_layout()
# plt.show()


## 4. Data Preprocessing


In [ ]:
# Handle missing values
# df = df.dropna()  # or use fillna()

# Feature scaling
# scaler = StandardScaler()
# features_scaled = scaler.fit_transform(df[['feature1', 'feature2', 'feature3']])

# print("Data preprocessing completed!")


## 5. Determine Optimal Number of Clusters


In [ ]:
# Elbow method to find optimal number of clusters
# inertias = []
# K_range = range(2, 11)

# for k in K_range:
#     kmeans = KMeans(n_clusters=k, random_state=42)
#     kmeans.fit(features_scaled)
#     inertias.append(kmeans.inertia_)

# # Plot elbow curve
# plt.figure(figsize=(10, 6))
# plt.plot(K_range, inertias, marker='o')
# plt.xlabel('Number of Clusters')
# plt.ylabel('Inertia')
# plt.title('Elbow Method for Optimal K')
# plt.grid(True)
# plt.show()


## 6. Apply K-Means Clustering


In [ ]:
# Apply K-Means with optimal number of clusters
# optimal_k = 4  # Adjust based on elbow method
# kmeans = KMeans(n_clusters=optimal_k, random_state=42)
# df['Cluster'] = kmeans.fit_predict(features_scaled)

# print(f"Customers segmented into {optimal_k} clusters!")
# print(df['Cluster'].value_counts())


## 7. Visualize Results


In [ ]:
# Dimensionality reduction for visualization
# pca = PCA(n_components=2)
# features_pca = pca.fit_transform(features_scaled)

# # Create scatter plot
# plt.figure(figsize=(12, 8))
# scatter = plt.scatter(features_pca[:, 0], features_pca[:, 1], 
#                       c=df['Cluster'], cmap='viridis', 
#                       alpha=0.6, s=50)
# plt.colorbar(scatter, label='Cluster')
# plt.xlabel('First Principal Component')
# plt.ylabel('Second Principal Component')
# plt.title('Customer Segmentation Visualization')
# plt.grid(True, alpha=0.3)
# plt.show()


## 8. Cluster Analysis


In [ ]:
# Analyze characteristics of each cluster
# cluster_summary = df.groupby('Cluster').mean()
# print(cluster_summary)

# # Visualize cluster characteristics
# cluster_summary.plot(kind='bar', figsize=(14, 6))
# plt.title('Average Characteristics by Cluster')
# plt.xlabel('Cluster')
# plt.ylabel('Average Value')
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.tight_layout()
# plt.show()


## 9. Conclusions and Next Steps

### Key Findings:
- Add your findings here

### Recommended Actions:
- List recommended actions for each customer segment

### Next Steps:
- Further analysis
- Model refinement
- Deployment considerations
